# Forex Options Portfolio Simulation and Evaluation

This notebook allows for interactive exploration and evaluation of the portfolio of European call options on the EUR/TND exchange rate. It demonstrates the pricing models implemented in this project and analyzes their performance.

In [ ]:
# Import necessary libraries
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yaml
from datetime import datetime

# Add parent directory to Python path to import project modules
sys.path.append('..')

# Import project modules
from src.data_generation.option_generator import OptionGenerator
from src.market_data.data_handler import MarketDataHandler
from src.portfolio.portfolio_manager import PortfolioManager
from src.evaluation.performance_metrics import calculate_model_comparison, rank_models
from src.visualization.plotting import (
    plot_spot_rates, plot_active_notional, plot_model_comparison, 
    plot_error_metrics, plot_option_distribution, plot_price_vs_strike,
    plot_volatility_smile, create_dashboard
)

# For better notebook display settings
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 6)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 200)

## 1. Load Configuration and Data

First, let's load the configuration file and the generated data.

In [ ]:
# Load configuration
with open('../config.yaml', 'r') as file:
    config = yaml.safe_load(file)

# Print some key configuration parameters
print(f"Currency Pair: {config['portfolio']['currency_pair']}")
print(f"Date Range: {config['time']['start_date']} to {config['time']['end_date']}")
print(f"Max Total Notional: €{config['portfolio']['max_total_notional']:,}")
print(f"Number of Options: {config['simulation']['num_options']}")

In [ ]:
# Load option contracts
options_file = "../data/generated/option_contracts.csv"
if os.path.exists(options_file):
    options_data = pd.read_csv(options_file)
    print(f"Loaded {len(options_data)} option contracts.")
else:
    print(f"Options file '{options_file}' not found. Generate data first.")
    # Generate option contracts
    generator = OptionGenerator(config_path='../config.yaml')
    options_data = generator.generate_portfolio()
    generator.save_portfolio_to_csv()
    print(f"Generated {len(options_data)} option contracts.")

# Load market data
market_handler = MarketDataHandler(config_path='../config.yaml')
market_data = market_handler.load_market_data()

if market_data[0] is None:
    print("Market data not found. Generating data...")
    market_data = market_handler.generate_market_data(save=True)
    print("Market data generated.")
else:
    print("Market data loaded successfully.")

# Unpack market data
spot_rates, volatility, interest_rates = market_data

## 2. Explore the Options Portfolio

Let's examine the structure and characteristics of the options portfolio.

In [ ]:
# Display the first few options
options_data.head()

In [ ]:
# Portfolio statistics
print(f"Total Notional: €{options_data['notional'].sum():,.2f}")
print(f"Average Notional per Option: €{options_data['notional'].mean():,.2f}")
print(f"Average Days to Maturity: {options_data['days_to_maturity'].mean():.1f} days")
print(f"Shortest Maturity: {options_data['days_to_maturity'].min()} days")
print(f"Longest Maturity: {options_data['days_to_maturity'].max()} days")

In [ ]:
# Visualize option distribution
fig = plot_option_distribution(options_data)
plt.show()

In [ ]:
# Load notional summary if available
notional_summary_file = "../data/generated/notional_summary.csv"
if os.path.exists(notional_summary_file):
    notional_summary = pd.read_csv(notional_summary_file)
    print(f"Loaded notional summary with {len(notional_summary)} days.")
    
    # Plot active notional over time
    fig = plot_active_notional(notional_summary)
    plt.show()

## 3. Explore Market Data

Let's examine the EUR/TND spot rates, volatility, and interest rates.

In [ ]:
# Plot spot rates and volatility
fig = plot_spot_rates(spot_rates, volatility)
plt.show()

In [ ]:
# Plot interest rates
plt.figure(figsize=(12, 6))
interest_rates['date'] = pd.to_datetime(interest_rates['date'])
plt.plot(interest_rates['date'], interest_rates['EUR_rate'] * 100, 'b-', label='EUR Interest Rate')
plt.plot(interest_rates['date'], interest_rates['TND_rate'] * 100, 'r-', label='TND Interest Rate')
plt.title('Interest Rates')
plt.xlabel('Date')
plt.ylabel('Rate (%)')
plt.grid(True)
plt.legend()
plt.show()

## 4. Price the Portfolio

Now, let's price the portfolio using all four pricing models:
1. Black-Scholes (Garman-Kohlhagen)
2. E-GARCH Monte Carlo
3. Merton Jump-Diffusion
4. SABR

In [ ]:
# Initialize portfolio manager
portfolio_manager = PortfolioManager(options_data, market_data, config_path='../config.yaml')

# Price portfolio using all models
priced_options = portfolio_manager.price_portfolio()

# Calculate risk metrics
priced_options, portfolio_risks = portfolio_manager.calculate_risks()

# Display the pricing results
priced_options[['option_id', 'notional', 'strike_price', 'days_to_maturity', 
                'bs_price', 'egarch_price', 'jd_price', 'sabr_price']].head()

In [ ]:
# Calculate the total option value according to each model
price_columns = ['bs_price', 'egarch_price', 'jd_price', 'sabr_price']
model_names = ['Black-Scholes', 'E-GARCH MC', 'Jump-Diffusion', 'SABR']

total_prices = {}
for col, name in zip(price_columns, model_names):
    total_prices[name] = (priced_options[col] * priced_options['notional']).sum()

# Print total portfolio values
for name, value in total_prices.items():
    print(f"{name}: €{value:,.2f}")

# Plot the comparison
fig = plot_model_comparison(priced_options, 'price')
plt.show()

In [ ]:
# Plot price vs. strike for each model
fig = plot_price_vs_strike(priced_options)
plt.show()

In [ ]:
# Plot the volatility smile
if 'implied_volatility' in priced_options.columns:
    fig = plot_volatility_smile(priced_options)
    plt.show()

In [ ]:
# Print portfolio risk metrics
for risk, value in portfolio_risks.items():
    print(f"{risk}: {value:,.2f}")

## 5. Calculate Actual Payoffs and PnL

Now, let's calculate the actual payoffs of the options at maturity and the resulting profit and loss (PnL) for each pricing model.

In [ ]:
# Calculate actual payoffs
options_with_payoffs = portfolio_manager.calculate_actual_payoffs(spot_rates)

# Calculate PnL
options_with_pnl, total_pnl = portfolio_manager.calculate_pnl()

# Display the results
options_with_pnl[['option_id', 'notional', 'strike_price', 'spot_rate_at_maturity', 
                  'actual_payoff', 'bs_price', 'bs_pnl', 'egarch_price', 'egarch_pnl', 
                  'jd_price', 'jd_pnl', 'sabr_price', 'sabr_pnl']].head()

In [ ]:
# Print total PnL for each model
for model, pnl in total_pnl.items():
    print(f"{model}: €{pnl:,.2f}")

# Plot the comparison
fig = plot_model_comparison(options_with_pnl, 'pnl')
plt.show()

## 6. Evaluate Model Performance

Finally, let's evaluate the performance of each pricing model using various metrics.

In [ ]:
# Evaluate model performance
metrics = portfolio_manager.evaluate_model_performance()

# Print metrics for each model
for model, model_metrics in metrics.items():
    print(f"\n{model.upper()} Model Metrics:")
    for metric, value in model_metrics.items():
        print(f"  {metric}: {value:,.4f}")

In [ ]:
# Calculate and rank model performance metrics
model_comparison = calculate_model_comparison(options_with_pnl)
ranked_models = rank_models(model_comparison)

# Display the comparison table
display(ranked_models)

In [ ]:
# Plot error metrics
if model_comparison is not None:
    fig = plot_error_metrics(model_comparison)
    plt.show()

## 7. Conclusion

This notebook has demonstrated the pricing, evaluation, and visualization of a portfolio of European call options on the EUR/TND exchange rate using four different pricing models. Each model has its strengths and weaknesses, as shown by the various performance metrics.

Key findings:
- The Black-Scholes (Garman-Kohlhagen) model provides a simple and computationally efficient solution, but it may not capture the volatility smile effect well.
- The E-GARCH Monte Carlo model captures the time-varying nature of volatility, including the asymmetric response to positive and negative returns shocks.
- The Merton Jump-Diffusion model accounts for sudden jumps in the exchange rate, which is important for capturing fat tails in the return distribution.
- The SABR model handles the stochastic nature of volatility and its correlation with the exchange rate, making it particularly suitable for capturing the volatility smile effect.

The choice of model should depend on the specific characteristics of the market and the options being priced.

In [ ]:
# Save the final results
output_dir = '../output'
os.makedirs(output_dir, exist_ok=True)
portfolio_manager.save_results(output_dir)
print(f"Results saved to {output_dir}")